# PHY5001 Assessment 2
##  Grover's algorithm 

This is the Jupyter Notebook which accompanies the pdf guide for Assignment 2, which focuses on Grover's Algorithm. Be advised that you should read the pdf as well, as there are questions which you will need to answer.

### Table of Contents
This hyperlinked table of contents may help reduce how much scrolling around you need to do to find the appropriate cells

- [Task 2: Selector](#Task2_cell)
    - [(a) GateSwitch](#Task2a_cell)
    - [(b) SelectorSix](#Task2b_cell)

    
- [Task 3: Amplifier](#Task3_cell)


- [Task 4: Putting together Grover's algorithm](#Task4_cell)


- [Task 5: Comparison of classical and quantum search](#Task5_cell)
    - [(a) RandomiseSix](#Task5a_cell)
    - [(b) Plotting comparison](#Task5Plot_cell)
    
    
- [Testing Functions](#Testing)
    - [Task 2 Testing](#Task2T_cell)
    - [Task 3 Testing](#Task3T_cell)
    - [Task 4 Testing](#Task4T_cell)
    - [Task 5 Testing](#Task5T_cell)

## Housekeeping
The cells below will import the appropriate packages you need for this assignment, assuming they are installed, and apply particular configuration settings for `DumpMachine`. You will need to run these cells each time you open this file to start work, as without them your functions below will not run as intended. 

In [7]:
import qsharp
import numpy as np
import matplotlib.pyplot as plt

In [8]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

In [9]:
%config dump.basisStateLabelingConvention="BitString" 
%config dump.phaseDisplayStyle="NumberOnly"

In [10]:
subscription_id = 'aa30f5b0-4da3-44b5-adaf-8b08b49065a9'
resource_group = 'AzureQuantum'
workspace_name = 'quantum-assignment'
location = 'East US'

In [11]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': None, 'manifest': None}

## Task 2: Selector

<a id='Task2_cell'></a>

In this task, you will write code to set up the **Selector** component of Grover's algorithm. You will need to write two functions to build this Selector:
 - A function **GateSwitch** which applies Pauli-X gates to qubits in a 6 qubit array, using a Boolean array input to configure which qubits will be modified by the Pauli-X gates
 - A function **SelectorSix** which takes a 6 qubit array input and:
   - applies GateSwitch
   - applies a controlled-Z gate to the entire array, with the 6th qubit in the array being the target
   - applies GateSwitch again


<a id='Task2a_cell'></a>
In the cell below, write your code for **GateSwitch**.

*Hint: You will likely need to include a `for` loop and an `if` condition to make this work.*

In [12]:
%%qsharp
operation GateSwitch(target: Qubit[], pattern: Bool[]) : Unit {
    // loop through each qubit and apply Puli-X gate if paterrn is true
    for i in 0..5 {
        if (pattern[i]){
            X(target[i]);
        }
    }
    
}

<a id='Task2b_cell'></a>
In the cell below, write your code for **SelectorSix**.

*Hint: Remember to use indices to indicate which qubit in the array you are referring to.*

In [13]:
%%qsharp
operation SelectorSix(target: Qubit[], pattern: Bool[]) : Unit {
    //step 1 : apply Gateswich which represent puli-X
    GateSwitch(target,pattern);
    //step 2 : apply controlled z in all qubit controlling tha last one 
    Controlled Z(target[0..4],target[5]);
    //step 3 : apply Gateswich to get the orginal state
    GateSwitch(target,pattern)
    
}

## Task 3: Amplifier

<a id='Task3_cell'></a>

In this task, you will write code to set up the **Amplifier** component of Grover's algorithm. You will need to write one function to build this Amplifier:
 - A function **AmplifySix** which applies which applies the appropriate gates to a 6 qubit array, with the target of the controlled-Z gate being the 6th qubit in the array.
 
 
In the cell below, write your code for **AmplifySix**

*Hint: Remember to use indices to indicate which qubit in the array you are referring to. In Workshops you have seen the tools needed for at least 3 different approaches to writing AmplifySix - you may find it useful to use two `for` loops for part of this, but do not have to use any*

In [14]:
%%qsharp
operation AmplifySix(target: Qubit[]) : Unit {
 // step 1 : apply hadamard and puli-x gate to each qubit 
   for i in 0..5 {
    H(target[i]);
    X(target[i]);
   }

 // step 2 : apply controlled-Z with the last qubit as the target
   Controlled Z(target[0..4],target[5]);

 // step 4 : apply puli-x and hadamard gate to each qubit 
   for i in 0..5 {
    X(target[i]);
    H(target[i]);
   }
 
}

<a id='Task4_cell'></a>
## Task 4: Putting together Grover's algorithm

In this task, you will write:
- A function **SixQGroverIteration** which will apply SelectorSix and AmplifySix as required for a single iteration of Grover's algorithm
- A function **SixQGrovers** which will apply SixQGroverIteration multiple times in a row, as determined by an input `repeats`

<a id='Task4a_cell'></a>
In the cell below, write your code for **SixQGroverIteration**.

In [15]:
%%qsharp
operation SixQGroverIteration(target: Qubit[], pattern: Bool[]) : Unit {
    // step 1: apply the selector to select the targe state 
    SelectorSix(target,pattern);
    
    // step 2 : apply the amplifier to boost the probability of the slected state 
    AmplifySix(target);

    
        
}

In the cell below, write your code for **SixQGrovers**

*Hint: One way to approach this is to write a `for` loop which will apply **SixQGroverIteration** on each repeat*

In [16]:
%%qsharp
operation SixQGrovers(target: Qubit[], pattern: Bool[], repeats: Int) : Unit {
    // This operation performs multiple iterations a 6-qubit array.

    for i in 0..repeats-1{
        SixQGroverIteration(target,pattern);
    }
       
    
}

<a id='Task5_cell'></a>
## Task 5: Comparison of classical and quantum search

Now that we have a function that performs a single iteration of Grover's algorithm, let's work towards comparing the performance of quantum search against classical search. To do so, you will need to write:
- A function **RandomiseSix**, which should use Hadamard gates on all 6 qubits in a 6 qubit input to create a completely randomised quantum superposition, in order to allow a fair test of the algorithm
- Plotting code for comparing the performance of classical and quantum search algorithms


<a id='Task5a_cell'></a>
In the cell below, write your code for **RandomiseSix**.

*Hint: Remember to use indices to indicate which qubit in the array you are referring to.*

In [17]:
%%qsharp
operation RandomiseSix(target: Qubit[]) : Unit {

    for X in 0..5 {
      H(target[X])     
    } 
    
}

The cell below contains a function which will combine your RandomiseSix with your SixQGrovers. This will allow you to simulate the action of Grover's algorithm for searching. Assuming you have checked that your functions above work correctly, this should not need any adjustments in order to run. 

In [18]:
%%qsharp
operation SixGroverRun(repeats: Int, pattern: Bool[]) : Result[] {
    use q = Qubit[6];
    RandomiseSix(q);
    SixQGrovers(q, pattern, repeats);
 return (MResetEachZ(q));
       
}

In the cell below:
- run a single iteration (eg. repeats = 1) of SixGroverRun for any 6 qubit Boolean pattern you choose - this will be fed into the selector. 
- Make note of the probability of measuring the state you have set the selector for.

Repeat the above, increasing the number of repeats from 1 to 20 in steps of 1, recording the probability of measuring the state you have selected at each point.

*Note: You may notice that the probability changes in an unexpected manner after 6 repeats - this is not an error!*

 after apllying the it in Quirk i gort thiese results---> the selector marked the state 010111, and the amplifier boosted its probability to 13.4827%, while other states remained at 1.3733%, confirming the correct state was amplified.

In [19]:
%%qsharp

// Example: Call SixGroverRun with 2 repeats and pattern 101010:
SixGroverRun(1, [true,false,true,false,true,false])



[One, Zero, Zero, Zero, One, Zero]

In [ ]:
import azure.quantum

operation = qsharp.compile("SixGroverRun(6, [true,false,true,false,true,false])")

workspace = azure.quantum.Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)
target = workspace.get_targets("quantinuum.sim.h1-1e")
job = target.submit(operation, "my-azure-quantum-job", shots=10 )
job.get_results()

................

{'[0, 0, 0, 0, 0, 1]': 0.1,
 '[1, 1, 0, 1, 0, 1]': 0.2,
 '[0, 0, 0, 1, 0, 1]': 0.1,
 '[0, 1, 1, 0, 0, 1]': 0.1,
 '[1, 1, 1, 1, 0, 1]': 0.1,
 '[0, 1, 0, 0, 0, 1]': 0.1,
 '[1, 1, 0, 1, 0, 0]': 0.1,
 '[1, 1, 1, 1, 1, 1]': 0.1,
 '[1, 0, 0, 1, 0, 1]': 0.1}